### Week 1 - Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

#### Webscrape Canada's list of postal codes from its wikipedia page

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

In [4]:
content_table = []

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['Postalcode'] = row.b.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')', ' ')).strip(' ')
        content_table.append(cell)

In [5]:
df = pd.DataFrame(content_table)
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                      'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                      'EtobicokeNorthwest':'Etobivoke Northwest',
                                      'East YorkEast Toronto':'East York/East Toronto',
                                      'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [7]:
print('Shape of the dataframe {}'.format(df.shape))

Shape of the dataframe (103, 3)


#### Order the dataframe to match the "Geospatial_Coordinates.csv" file

In [8]:
order_df = df.sort_values(by=['Postalcode'], ascending = True)
order_df.reset_index(drop = True, inplace = True)
order_df.head(5)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.reset_index(drop = True, inplace = True)

In [11]:
Toronto_df = pd.concat([order_df, coordinates_df], axis = 1)
Toronto_df.drop(['Postal Code'], axis = 1, inplace = True)
Toronto_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Narrowing my clustering/segmentation task to the Borough with the most Neighborhoods

In [22]:
Toronto_df.groupby('Borough').count()

,Postalcode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,17,17,17,17
Downtown Toronto Stn A,1,1,1,1
East Toronto,4,4,4,4
East Toronto Business,1,1,1,1
East York,4,4,4,4
East York/East Toronto,1,1,1,1
Etobicoke,11,11,11,11
Etobivoke Northwest,1,1,1,1


In [13]:
northYork_df = Toronto_df[Toronto_df['Borough'] == 'North York'].reset_index(drop = True)
northYork_df.head(24)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [14]:
geolocator = Nominatim(user_agent = 'tor_explorer')
location = geolocator.geocode('North York, TOR')
latitude = location.latitude
longitude = location.longitude
print('Coordinates for North York in Toronto are the following: {} / {}'.format(latitude, longitude))

Coordinates for North York in Toronto are the following: 43.7627912 / -79.4064452


In [19]:
map_northYork = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, label in zip(northYork_df['Latitude'], northYork_df['Longitude'], northYork_df['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 4,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_northYork)
    
map_northYork

#### Initiate Foursquare

In [ ]:
CLIENT_ID = 'NKIICUFP3F2YR2CNU31ATDOOGFMZE2MDEHC3ZORF3C0EUSUI'
CLIENT_SECRET = 'WMNZ2GATYAXECUDL4YZX0AEIMUBY4EHYNMYKALOU0FY3QAZR'
VERSION = '20200615'
LIMIT = 100